In [4]:
import camelot

In [5]:
import glob

In [6]:
directory = '/Users/joqsan/0-Excel/Reading_From_Folders/CCAF/*.pdf'

In [7]:
directory

'/Users/joqsan/0-Excel/Reading_From_Folders/CCAF/*.pdf'

In [8]:
files = [filename for filename in glob.glob(directory)]

In [9]:
files

['/Users/joqsan/0-Excel/Reading_From_Folders/CCAF/CCAF_03.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/CCAF/CCAF_02.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/CCAF/CCAF_01.pdf']

## Let's test CCAF

In [10]:
import camelot
import pandas as pd
%matplotlib notebook
import re
import calendar
import locale
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')

'es_ES.UTF-8'

In [11]:
months = list(calendar.month_abbr)
months = list(map(lambda x: x.capitalize(), months))

In [12]:
area = ['16, 466, 393, 298', '393, 471, 736, 382']
cols = ['575.2, 418.8, 711.4', '141.3, 184.8, 303, 362.1']

In [38]:
def transpose_data(df, month, name, drop_index=False):
    df = df.transpose()
    df.columns = df.iloc[0, :]
    df = df.drop(0, axis=0)
    
    df = df.set_index([[month], [name]])
    
    if drop_index:
        df.index = df.index.droplevel(level=1)
    
    return df

In [43]:
def ccaf_data_cleaning(dfs):
    
    part1 = dfs[0].df.copy()
    month = part1.iloc[4, 0].split(' ')[0][:3]
    
    part2 = dfs[1].df.copy()
    part2.drop([0,1, 7], inplace=True)
    part2.loc[[6, 13], 0:2] = part2.loc[[6, 13], 0:2].shift(-2, axis=1)
    part2.iloc[:, 0] = part2.iloc[:, 0].str.title()
    part2 = part2.iloc[:, [0, 4]].copy()
    part2.iloc[-1, 0] = 'Total A Pagar A Caja'
    
    part2.iloc[:, [1]] = part2.iloc[:, [1]].apply(lambda x: x.str.replace('.', '')).astype(int)
    
    ccaf = transpose_data(part2, month, 'CCAF', drop_index=True)
    
    return ccaf

In [44]:
ccaf_list = []
for ccaf_file in files:
    ccaf_dfs = camelot.read_pdf(ccaf_file, pages='1', flavor='stream', table_areas=area, columns=cols)
    ccaf = ccaf_data_cleaning(ccaf_dfs)
    ccaf_list.append(ccaf)

In [45]:
df = pd.concat(ccaf_list)

In [46]:
df

,Cotización No Afiliados A Isapre,Asignación Familiar,Asignación Familiar Retroactiva,Reintegros De Asignación Familiar,Cotización - Rebajas,Creditos Personales,Convenios Dentales,Leasing,Seguros De Vida,Otros,Total Productos,Total A Pagar A Caja
Mar,102176,119900,0,0,-17724,0,0,0,0,0,0,-17724
Feb,89605,9180,0,0,80425,0,0,0,0,0,0,80425
Ene,84862,9180,19775,0,55907,66921,0,0,0,0,66921,122828


In [48]:
df['Asignación Familiar'] + df['Asignación Familiar Retroactiva']

Mar    119900
Feb      9180
Ene     28955
dtype: object

In [52]:
# Column O: df['Asignación Familiar'] + df['Asignación Familiar Retroactiva'] 
# Column S: df['Asignación Familiar']
# What is the difference between CCAF Asignacion Familiar (column O) and 0.6% (column S, 
#                                                     which is also Asignacion Familiar)
#df['Prestamo CCAF'] = Creditos Personales  + Convenios Dentales + Leasing + Seguros De Vida + Otros?